In [1]:
import numpy as np

from ml.my_k_means import my_k_means
from models.DevelopmentSession import DevelopmentSession
from db.sqlite import SQLiteManager
from models.enums import ErrorType
from utils.plotHelper import plot_radar_node, plot_error_class, plot_radar_all_centroid_same_plot, plot_radar_all_centroid_different_plot, plot_radar_ahp
import sys
import pandas as pd

In [3]:
# * 1) Connect with Database
SQLiteManager.connect()
cur = SQLiteManager.conn
sql = 'select multiple_ses_stud.student, exams.id, sessions.date, multiple_ses_stud.cnt, results.grade, grade_sort.id as grd'
sql += ' from results, sessions, multiple_ses_stud, students, exams, grade_sort' 
sql += ' where results.grade=grade_sort.grade and exams.session=sessions.id and' 
sql += ' students.matr like multiple_ses_stud.student and exams.student=students.id and'
sql += ' results.session like sessions.date and results.student like students.matr'
sql += ' order by multiple_ses_stud.student, exams.id'

df = pd.read_sql_query(sql,SQLiteManager.conn)
print(df.head())



     student    id      date  cnt  grade  grd
0  832000811   654  20180321    9  INSUF    6
1  832000811   711  20180725    9    DIS    4
2  832000811  1101  20170717    9  INSUF    6
3  A13000318   542  20181011    8    DIT    2
4  A13000318   704  20180725    8  INSUF    6


In [42]:
df['ahp'] = np.zeros(df.shape[0])-1



In [52]:
ahp_weigths = pd.read_csv("ahp_result.csv", header=None )
ahp_weigths['ahp']=ahp_weigths.iloc[:,1:9].sum(axis=1)
ahp_weigths=ahp_weigths.drop(ahp_weigths.columns[1:9], axis=1)
ahp_weigths.columns=['id','ahp']
print(ahp_weigths.head())


   id       ahp
0   8  0.028330
1  11  0.007572
2  18  0.016956
3  25  0.017713
4  27  0.030041


In [70]:

ahp_ids = ahp_weigths.iloc[:, 0].values
for i in range(df.shape[0]):
    if df.iloc[i]['id'] in ahp_ids:
        # print(df.iloc[i]['id'], ahp_weigths[ahp_weigths['id']==df.iloc[i]['id']]['ahp'])
        df.at[i,'ahp'] = ahp_weigths[ahp_weigths['id']==df.iloc[i]['id']]['ahp']
        # print(df.iloc[i])
df = df[df['ahp']>=0]
print(df.head())

      student    id      date  cnt  grade  grd       ahp
10  A13000758   936  20190709    2    SUF    5  0.010900
14  A13000934   912  20181108    2  INSUF    6  0.013884
20  A13001074   605  20180613    5  INSUF    6  0.042955
47  A13001300   981  20180124    5  INSUF    6  0.024264
48  A13001300  1030  20171204    5  INSUF    6  0.003428


In [95]:
df_min = df.groupby('student').agg('min').sort_values(by='student')
print(df_min.shape)
for i in range(df_min.shape[0]):
    student = df[df['student']==df_min.iloc[i].name]
    if student.shape[0] >1:
        student['dd'] = pd.to_datetime(student['date'], format='%Y%m%d') 
        student = student.sort_values(by='dd')
        print(student)



(75, 6)
      student    id      date  cnt  grade  grd       ahp         dd
49  A13001300  1084  20170616    5    RIT    7  0.017713 2017-06-16
48  A13001300  1030  20171204    5  INSUF    6  0.003428 2017-12-04
47  A13001300   981  20180124    5  INSUF    6  0.024264 2018-01-24
      student    id      date  cnt  grade  grd       ahp         dd
62  A13001314  1103  20170717    5  INSUF    6  0.015942 2017-07-17
61  A13001314  1003  20171023    5    RIT    7  0.053140 2017-10-23
      student    id      date  cnt  grade  grd       ahp         dd
83  A13001354  1106  20170717    6  INSUF    6  0.045107 2017-07-17
82  A13001354  1031  20171204    6  INSUF    6  0.031497 2017-12-04
       student    id      date  cnt  grade  grd       ahp         dd
111  A13001406  1124  20170920    2  INSUF    6  0.022198 2017-09-20
110  A13001406  1005  20171023    2    SUF    5  0.014493 2017-10-23
       student    id      date  cnt  grade  grd       ahp         dd
120  A13001424  1100  20170717   13 

/tmp/ipykernel_333197/485618572.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student['dd'] = pd.to_datetime(student['date'], format='%Y%m%d')
/tmp/ipykernel_333197/485618572.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student['dd'] = pd.to_datetime(student['date'], format='%Y%m%d')
/tmp/ipykernel_333197/485618572.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu